In [ ]:
#Use this code whenever we've updated the environment.yml document
#%conda env update --file '../environment.yml'  --prune

In [ ]:
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np
import os
from trimesh import Trimesh

from skimage import morphology
from skimage.segmentation import expand_labels, watershed
from scipy.ndimage import convolve1d
from skimage import measure


from dask_image import imread
from dask_image import ndmeasure
from dask_image import ndfilters
from dask import array as da
from dask import delayed
from dask.distributed import Client, progress

import napari

In [ ]:
client = Client(processes=False, threads_per_worker=6,
                n_workers=4, memory_limit='12GB')
client

In [ ]:
specific = 'F231006L'
finger_dir = {1: '../fingers/index/full/' + specific + '/',
              2: '../fingers/middle/full/' + specific + '/',
              3: '../fingers/ring/full/' + specific + '/',
              4: '../fingers/pinky/full/' + specific + '/',
             }
spacing = np.load('../spacing/' + specific + '.npy')

In [ ]:
i = 4
#If the file is too big, then delete the ".compute" part below.
finger = da.from_npy_stack(finger_dir[i])

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(
    finger,
    scale = spacing
)

In [ ]:
smoothed = ndfilters.median_filter(
    image = finger,
    size = 3,
).compute()

In [ ]:
viewer.add_image(
    smoothed,
    name = 'smoothed',
    scale = spacing,
)

In [ ]:
#Not sure what threshold to set this at
threshold = 450
verts, faces, normals, values = measure.marching_cubes(
    volume = smoothed,
    level = threshold,
    spacing = spacing,
)

In [ ]:
mesh = Trimesh(vertices=verts, faces=faces)

In [ ]:
mesh_view = viewer.add_surface(
    data = (verts, faces, values),
    scale = spacing,
)

In [ ]:
verts